In [105]:
import pandas as pd
import json
import os
import numpy as np
import time
from datetime import timedelta

import importlib

from pymongo import MongoClient

client = MongoClient('localhost', 27017) # change the ip and port to your mongo database's
db = client['kagglecrawler']
search_log_coll = db['search_log']
download_log_coll = db['download_log']
raw_notebook_coll = db['raw_notebooks']
task_log_coll = db['task_log']
central_task_log_coll = db['central_task_log']
notebook_metadata_coll = db['notebook_metadata_coll']
QUERY_FILE = os.path.join(os.getcwd(), 'Queries/pwc_queries.csv')


In [106]:
from kagglecrawler.distributed_notebook_crawling import KaggleNotebookCrawler

# Create crawler
kwargs = {
    'search_log_coll': search_log_coll, 
    'download_log_coll': download_log_coll, 
    'central_log_coll': download_log_coll, 
    'task_log_coll': task_log_coll, 
    'raw_notebook_coll': raw_notebook_coll, 
    'notebook_metadata_coll': notebook_metadata_coll
}
crawler = KaggleNotebookCrawler(**kwargs)

In [107]:
crawler.notebook_metadata_coll

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'kagglecrawler'), 'notebook_metadata_coll')

In [113]:
if not list(central_task_log_coll.find()): 
    # df_pwc_queries = pd.read_csv(QUERY_FILE)
    # df_tasks = pd.DataFrame(df_pwc_queries['query'])
    # df_tasks['searched'] = 0
    # df_tasks['downloaded'] = 0
    records = df_tasks.to_dict('records')
    crawler.update_task_log(records, central_task_log_coll)
    print('updated')


updated


In [116]:
df_tasks = pd.DataFrame.from_dict(list(central_task_log_coll.find()))
df_tasks

,_id,query,searched,downloaded
0,6387a608727ddf9f05e7e9d4,MinCut Pooling,1,1
1,6387a608727ddf9f05e7e9d6,Residual GRU,82,82
2,6387a608727ddf9f05e7e9d8,Four-dimensional A-star,55,55
3,6387a608727ddf9f05e7e9da,Hierarchical Network Dissection,0,0
4,6387a608727ddf9f05e7e9dc,DCLS,4,4
...,...,...,...,...
14160,6387a63f727ddf9f05e85895,Citation Intent Classification,-1,-1
14161,6387a63f727ddf9f05e85897,Extractive Text Summarization,-1,-1
14162,6387a63f727ddf9f05e85899,Cloud Detection,-1,-1
14163,6387a63f727ddf9f05e8589b,Problem-Solving Deliberation,-1,-1


In [117]:
un_searched = df_tasks.loc[df_tasks.searched==-1]
un_searched

,_id,query,searched,downloaded
1541,6387a60a727ddf9f05e7f5df,Gravity,-1,-1
1559,6387a60a727ddf9f05e7f603,Discrete Cosine Transform,-1,-1
1565,6387a60a727ddf9f05e7f60f,Gated Linear Network,-1,-1
1568,6387a60a727ddf9f05e7f615,Weight Decay,-1,-1
1569,6387a60a727ddf9f05e7f617,BiGG,-1,-1
...,...,...,...,...
14160,6387a63f727ddf9f05e85895,Citation Intent Classification,-1,-1
14161,6387a63f727ddf9f05e85897,Extractive Text Summarization,-1,-1
14162,6387a63f727ddf9f05e85899,Cloud Detection,-1,-1
14163,6387a63f727ddf9f05e8589b,Problem-Solving Deliberation,-1,-1
